In [20]:
import os
import numpy as np
import time
import copy
import sys

import matplotlib
import matplotlib.pyplot as plt

ang_2_bohr = 1.0/0.52917721067
hart_2_ev = 27.21138602

import cp2k_utilities as cu

In [21]:
folder = "/home/kristjan/local_work/asetk_example1/for_restart_file/"
file_basis_set = folder + "BR"
file_xyz = folder + "anthracene.xyz"
file_restart = folder + "ANTHRACENE-RESTART.wfn"
file_cp2k_inp = folder + "cp2k.inp"
file_cp2k_out = folder + "cp2k.out"


#folder = "/home/kristjan/sshfs/marconi_scratch/cp2k_si_morb/"
#file_basis_set = folder + "BR"
#file_xyz = folder + "Si.xyz"
#file_restart = folder + "PROJ-RESTART.wfn"
#file_cp2k_inp = folder + "cp2k.inp"
#file_cp2k_out = folder + "out.log"


#folder = "/home/kristjan/sshfs/daint_scratch/cp2k_c2h2_morbs/morbs_added_mos_cart/"
#file_basis_set = folder + "BR"
#file_xyz = folder + "p.xyz"
#file_restart = folder + "PROJ-RESTART.wfn"
#file_cp2k_inp = folder + "cp2k.inp"
#file_cp2k_out = folder + "out"

In [22]:
emin = -4.0
emax =  0.0

In [23]:
time1 = time.time()
elem_basis_names, cell = cu.read_cp2k_input(file_cp2k_inp)
print("Read cp2k input: %.3f" % (time.time()-time1))

time1 = time.time()
fermi = cu.read_fermi_from_cp2k_out(file_cp2k_out)
print("Read cp2k out: %.3f" % (time.time()-time1))

time1 = time.time()
at_positions, at_elems = cu.read_atoms(file_xyz)
print("Read xyz: %.3f" % (time.time()-time1))

time1 = time.time()
basis_sets = cu.read_basis_functions(file_basis_set, elem_basis_names)
print("Read basis sets: %.3f" % (time.time()-time1))

time1 = time.time()
morb_composition, morb_energies, morb_occs, ref_en = cu.load_restart_wfn_file(file_restart, emin, emax, fermi)
print("Found %d orbitals" % len(morb_energies))
print("Read restart: %.3f" % (time.time()-time1))

Read cp2k input: 0.001
Read cp2k out: 0.001
Read xyz: 0.001
Read basis sets: 0.009
First molecular index in energy range:  25
Found 8 orbitals
Read restart: 0.008


In [24]:
i_homo = 0
for i, occ in enumerate(morb_occs):
    if occ < 1.0:
        i_homo = i - 1
        break
    i_homo = i
print(i_homo)

7


In [25]:
# Define real space grid
# Cp2k chooses close to 0.08 angstroms (?)
step = 0.08 # Good to choose a factor of the lattice parameter 4.26
step *= ang_2_bohr
cell_n = (np.round(cell/step)).astype(int)

# !!! NB: Redefine cell such that step stays the same (and a factor of lattice param)
#cell = cell_n*step

#cu.center_atoms_to_cell(at_positions, cell)

In [26]:
time1 = time.time()

dv = cell/cell_n
x_arr = np.arange(0, cell[0], dv[0])
y_arr = np.arange(0, cell[1], dv[1])
z_arr = np.arange(0, cell[2], dv[2])
x_grid, y_grid, z_grid = np.meshgrid(x_arr, y_arr, z_arr, indexing='ij')

# Define small grid for orbital evaluation
# and convenient PBC implementation
loc_cell = np.array([16.0, 16.0, 16.0])*ang_2_bohr
x_arr_loc = np.arange(0, loc_cell[0], dv[0])
y_arr_loc = np.arange(0, loc_cell[1], dv[1])
z_arr_loc = np.arange(0, loc_cell[2], dv[2])
loc_cell_n = np.array([len(x_arr_loc), len(y_arr_loc), len(z_arr_loc)])
# Define it such that the origin is somewhere
# in the middle but exactly on a grid point
mid_ixs = (loc_cell_n/2).astype(int)
x_arr_loc -= x_arr_loc[mid_ixs[0]]
y_arr_loc -= y_arr_loc[mid_ixs[1]]
z_arr_loc -= z_arr_loc[mid_ixs[2]]
x_grid_loc, y_grid_loc, z_grid_loc = np.meshgrid(x_arr_loc, y_arr_loc, z_arr_loc, indexing='ij')

# Some info
print("Main cell:   ", cell, cell_n)
print("Local cell: ", loc_cell, loc_cell_n)

morb_grids = 0 # release memory from previous run (needed in some rare cases)
morb_grids = [np.zeros(cell_n) for _ in range(len(morb_composition))]

print("---- Setup: %.4f" % (time.time() - time1))

time_radial_calc = 0.0
time_spherical = 0.0
time_loc_glob_add = 0.0


for i_at in range(len(at_positions)):
    elem = at_elems[i_at][0]
    pos = at_positions[i_at]

    # how does the position match with the grid?
    int_shift = (pos/dv).astype(int)
    frac_shift = pos/dv - int_shift

    # Shift the local grid such that origin is on the atom
    x_grid_rel_loc = x_grid_loc - frac_shift[0]*dv[0]
    y_grid_rel_loc = y_grid_loc - frac_shift[1]*dv[1]
    z_grid_rel_loc = z_grid_loc - frac_shift[2]*dv[2]

    r_vec_2 = x_grid_rel_loc**2 + y_grid_rel_loc**2 + z_grid_rel_loc**2

    aorbs = []

    for i_shell, shell in enumerate(basis_sets[elem]):
        l = shell[0]
        es = shell[1]
        cs = shell[2]

        # Calculate the radial part of the atomic orbital
        time2 = time.time()
        radial_part = np.zeros(loc_cell_n)
        for e, c in zip(es, cs):
            radial_part += c*np.exp(-1.0*e*r_vec_2)
        time_radial_calc += time.time() - time2

        for i, m in enumerate(range(-l, l+1, 1)):
            time2 = time.time()
            atomic_orb = radial_part*cu.spherical_harmonic_grid(l, m,
                                                             x_grid_rel_loc,
                                                             y_grid_rel_loc,
                                                             z_grid_rel_loc)
            time_spherical += time.time() - time2

            aorbs.append(atomic_orb)

            for i_mo in range(len(morb_composition)):
                i_set = 0 # SHOULD START SUPPORTING MULTIPLE SET BASES AT SOME POINT
                coef = morb_composition[i_mo][i_at][i_set][i_shell][i]
            
                # Add the atomic orbital on the local grid to the global grid
                origin_diff = int_shift - mid_ixs
                time2 = time.time()
                cu.add_local_to_global_grid(coef*atomic_orb, morb_grids[i_mo], origin_diff)
                time_loc_glob_add += time.time() - time2

print("---- Radial calc time : %4f" % time_radial_calc)
print("---- Spherical calc time : %4f" % time_spherical)
print("---- Loc -> glob time : %4f" % time_loc_glob_add)
print("---- Total time: %.4f"%(time.time() - time1))         

Main cell:    [ 28.34589188  37.79452251  28.34589188] [188 250 188]
Local cell:  [ 30.23561801  30.23561801  30.23561801] [201 200 201]
---- Setup: 0.2077
---- Radial calc time : 135.835724
---- Spherical calc time : 9.885761
---- Loc -> glob time : 106.584825
---- Total time: 255.8387


In [18]:
cu.write_cube_file("/home/kristjan/local_work/asetk_example1/for_restart_file/test.cube",
                   file_xyz, cell, cell_n, morb_grids[i_homo-5])

In [19]:
# check for normalization
dv = cell/cell_n
vol_elem = dv[0]*dv[1]*dv[2]
np.sum(morb_grids[i_homo]**2)*vol_elem

0.99999971559830036